In [23]:
%matplotlib qt
import numpy as np
import os.path as op
from pandas import read_csv
import mne
from mne.io import read_raw_fif
from mne.datasets import visual_92_categories
from neurora.nps_cal import nps
from neurora.rdm_cal import eegRDM
from neurora.rdm_corr import rdm_correlation_spearman
from neurora.corr_cal_by_rdm import rdms_corr
from neurora.rsa_plot import plot_rdm, plot_corrs_by_time, plot_nps_hotmap, plot_corrs_hotmap
import pandas as pd

from neurora.rsa_plot import plot_tbytsim_withstats

In [24]:
list_epochs_all = list()


# DD
# 去掉 20数据不全 24没数据 25无法ICA
sub_ids = [

    "pre4008_y","pre4009_y","pre4010_y","pre4011_y","pre4012_y","pre4013_y",
    "pre4014_y","pre4015_y","pre4016_y","pre4017_y", "pre4018_y","pre4019_y",
    "pre4021_y","pre4022_y","pre4023_y","pre4026_y",
    "pre4027_y","pre4028_y","pre4029_y","pre4030_y","pre4031_y","pre4032_y",
    "pre4033_y","pre4034_y",

]
sub_ids = [

    "pre4008_y","pre4009_y","pre4010_y","pre4011_y","pre4012_y","pre4013_y",
    "pre4014_y","pre4015_y","pre4016_y","pre4017_y", "pre4018_y","pre4019_y",
    "pre4021_y","pre4022_y","pre4023_y","pre4026_y",
    "pre4027_y","pre4028_y","pre4029_y","pre4030_y"
]
# DD组
sub_ids = [
            'pre4010_y','pre4012_y','pre4013_y','pre4015_y','pre4017_y','pre4018_y','pre4019_y','pre4021_y','pre4022_y','pre4023_y','pre4026_y','pre4027_y','pre4030_y',
          ]
# 非DD组
# sub_ids = [
#            'pre4008_y','pre4009_y','pre4011_y','pre4014_y','pre4028_y','pre4029_y'
#          ]


# # 正常儿童
# sub_ids = [
#
#      "pre4003_y","pre4007_y","pre4008_y","pre4009_y","pre4010_y","pre4011_y","pre4012_y","pre4013_y"
#
# ]

file_path = 'E:\实验\阅读困难\数据处理\pre4\data\\6epoch_clean_RSA\\'
channels_nieye = ['M1','T7','P7','TP7','M2','T8','P8','TP8','CP5','CP6','C5','C6']
channels_nieye = ['T7','P7','TP7','M2','T8','P8','TP8','CP5','CP6','C5','C6']
# 左右脑 偏侧化.
channels_left = ['Fp1',  'AF3',  'F7', 'F5', 'F3', 'F1',  'FT7', 'FC5', 'FC3', 'FC1',  'T7', 'C5', 'C3', 'C1',   'TP7', 'CP5', 'CP3', 'CP1',  'P7', 'P5', 'P3', 'P1',   'PO7', 'PO5', 'PO3',  'O1', ]

channels_right = ['Fp2','AF4',  'F2', 'F4', 'F6', 'F8', 'FC2', 'FC4', 'FC6', 'FT8','C2', 'C4', 'C6', 'T8', 'CP2', 'CP4', 'CP6', 'TP8', 'M2', 'P2', 'P4', 'P6', 'P8','PO4', 'PO6', 'PO8','O2',]

channels_all = ['Fp1', 'Fpz', 'Fp2', 'AF3', 'AF4', 'F7', 'F5', 'F3', 'F1', 'Fz', 'F2', 'F4', 'F6', 'F8', 'FT7', 'FC5', 'FC3', 'FC1', 'FCz', 'FC2', 'FC4', 'FC6', 'FT8', 'T7', 'C5', 'C3', 'C1', 'Cz', 'C2', 'C4', 'C6', 'T8', 'TP7', 'CP5', 'CP3', 'CP1', 'CPz', 'CP2', 'CP4', 'CP6', 'TP8', 'M2', 'P7', 'P5', 'P3', 'P1', 'Pz', 'P2', 'P4', 'P6', 'P8', 'PO7', 'PO5', 'PO3', 'POz', 'PO4', 'PO6', 'PO8', 'O1', 'Oz', 'O2', ]


channels_all = channels_right
num_channel = len(channels_all)

for sub_id in sub_ids:
    data_path = file_path + sub_id + '_RSA-epo.fif'

    epochs_all = mne.read_epochs(fname=data_path)
    # 如果只想看某一部分的脑区,配合更改ndarray的维度
    epochs_all = epochs_all.pick(picks=channels_all)

    list_epochs_all.append(epochs_all)

'''
思路，先求ERP，然后再给ndarray，然后再计算RDM
'''

Reading E:\实验\阅读困难\数据处理\pre4\data\6epoch_clean_RSA\pre4010_y_RSA-epo.fif ...
    Found the data of interest:
        t =    -200.00 ...     800.00 ms
        0 CTF compensation matrices available
Not setting metadata
1159 matching events found
No baseline correction applied
0 projection items activated
Reading E:\实验\阅读困难\数据处理\pre4\data\6epoch_clean_RSA\pre4012_y_RSA-epo.fif ...
    Found the data of interest:
        t =    -200.00 ...     800.00 ms
        0 CTF compensation matrices available
Not setting metadata
1141 matching events found
No baseline correction applied
0 projection items activated
Reading E:\实验\阅读困难\数据处理\pre4\data\6epoch_clean_RSA\pre4013_y_RSA-epo.fif ...
    Found the data of interest:
        t =    -200.00 ...     800.00 ms
        0 CTF compensation matrices available
Not setting metadata
1270 matching events found
No baseline correction applied
0 projection items activated
Reading E:\实验\阅读困难\数据处理\pre4\data\6epoch_clean_RSA\pre4015_y_RSA-epo.fif ...
    Found t

'\n思路，先求ERP，然后再给ndarray，然后再计算RDM\n'

In [25]:

'''
生成每种条件对应的evoked
而且是按照次序
'''
num_channel = len(channels_all)
conds = ['101', '102', '103', '104', '111', '112', '113', '114', '121', '122', '123', '124', '131', '132', '133', '134', ]

eegdata = np.zeros([len(sub_ids),16, num_channel, 1001], dtype=np.float32)
#初始值
subindex = 0
for m in range(len(sub_ids)):
    #单个被试的数据
    subdata = np.zeros([16, num_channel, 1001], dtype=np.float32)
    for i in range(len(conds)):
        epochs = list_epochs_all[m][conds[i]]
        evoked = epochs.average()
        data = evoked.data
        subdata[i] = data
    eegdata[m] = subdata

In [26]:
# shape of megdata: [n_subs, n_cons, n_chls, n_ts] -> [n_cons, n_subs, n_chls, n_ts]
eegdata = np.transpose(eegdata, (1, 0, 2, 3))

# shape of megdata: [n_cons, n_subs, n_chls, n_ts] -> [n_cons, n_subs, n_trials, n_chls, n_ts]
# here data is averaged, so set n_trials = 1
eegdata = np.reshape(eegdata, [16, len(sub_ids), 1, num_channel, 1001])


In [27]:
# Calculate the RDM based on the data during 190ms-210ms
#sub_opt=1 indicate不止一个被试。
#The shape of EEGdata must be [n_cons, n_subs, n_trials, n_chls, n_ts].
# Sometimes, the numbers of trials under different conditions are not same. In NeuroRA, we recommend users to average
#     the trials under a same condition firstly in this situation. Thus, the shape of input (EEG_data) should be
#     [n_cons, n_subs, 1, n_chls, n_ts].


rdm = eegRDM(eegdata[:, :, :, :, 340:350])
# Plot this RDM
plot_rdm(rdm[0], percentile=True)


Computing RDMs

RDMs computing finished!


0

## Calculate the RDMs

In [28]:
# Calculate the RDMs by a 10ms time-window
# (raw sampling requency is 1000Hz, so here time_win=10ms/(1s/1000Hz)/1000=10)
# sub_opt = 1

# 默认窗口是50，那么就是 （1000-50）/5 = 190
#

rdms = eegRDM(eegdata, time_opt=1, time_win=50, time_step=5, sub_opt=1)


Computing RDMs
Calculating: [====================================================================================================] 100.00%
RDMs computing finished!


In [29]:
# Plot the RDM of -100ms, 0ms, 50ms, 100ms, 150ms, 200ms
times = [0, 10, ]
for t in times:
    plot_rdm(rdms[1,t], percentile=True)

## 辅音_理论模型


In [30]:

'''
辅音_理论模型
'''

RDM_model_fuyin = np.zeros([16,16], dtype=np.float32)
# for i in range(20):
#     for j in range(20):
#         if abs(i-j)<=3:
#             RDM_model_fuyin[i,j] = 0
#         else:
#             RDM_model_fuyin[i,j] =
zeroM = np.zeros([4,4], dtype=np.float32)
oneM =  np.ones((4, 4), dtype=np.float32)

a1=np.concatenate((zeroM,oneM,oneM,oneM),axis = 1)
a2=np.concatenate((oneM,zeroM,oneM,oneM),axis = 1)
a3=np.concatenate((oneM,oneM,zeroM,oneM),axis = 1)
a4=np.concatenate((oneM,oneM,oneM,zeroM),axis = 1)

RDM_model_fuyin = np.concatenate((a1,a2,a3,a4),axis=0)
plot_rdm(RDM_model_fuyin, percentile=True)

0

In [31]:

# （sub,times,2）
corrs_EEGvsFuyin = rdms_corr(RDM_model_fuyin, rdms)
%matplotlib qt
# Plot the corrs1
# corrs1 = np.reshape(corrs1, [1, 191, 2])
# plot_corrs_by_time(corrs1, time_unit=[-0.2, 0.005])
from neurora.rsa_plot import plot_tbytsim_withstats
from neurora.stuff import clusterbased_permutation_1d_1samp_1sided

#配合RDMS的时间窗是 50ms，步长是5ms = 0.005
# start_time =
ps_EEGvsFuyin =  plot_tbytsim_withstats(corrs_EEGvsFuyin, start_time=-0.15, end_time=0.805, time_interval=0.005, p=0.05,clusterp=0.05,stats_time=[0,0.8],color='b', xlim=(-0.2,1), fontsize=9)

# plot_tbytsim_withstats 内嵌了  clusterbased_permutation_1d_1samp_1sided,统计部分由这个函数来完成的.
# similarities = corrs_EEGvsFuyin[:, :, 0]
# clusterbased_permutation_1d_1samp_1sided(similarities)



Computing similarities

Computing finished!

Permutation test
Calculating: [====================================================================================================] 100.00%
Cluster-based permutation test finished!


Significant time-windows:
305ms to 400ms
540ms to 625ms


### 保存结果

In [11]:
'''
保存corrs_EEGvsFuyin

1 需要算出极大值和对应的时间点-算峰值的方法 ---为了和行为做相关.

2 算显著时间点均值

'''

# 问题:这里应该保存 -200 - 800ms的 还是保存 显著的时间窗口内的呢?
# 应该保存窗口内的,不然没有做的意义了.
# idea 可以用 ps来进行窗口的限制 截取对应的时间窗
np.save('./脑电结果/corrs_EEGvsFuyin_DD_颞叶.npy', corrs_EEGvsFuyin)
np.save('./脑电结果/ps_EEGvsFuyin_DD_颞叶.npy', ps_EEGvsFuyin)


## 音调_理论模型

In [32]:

'''
音调 模型
'''

RDM_model_yindiao = np.zeros([16,16], dtype=np.float32)


oneM =  np.ones((4, 4), dtype=np.float32)
for i in range(4):
    oneM[i][i] = 0
zeroM = oneM

a1=np.concatenate((zeroM,oneM,oneM,oneM,),axis = 1)
a2=np.concatenate((oneM,zeroM,oneM,oneM,),axis = 1)
a3=np.concatenate((oneM,oneM,zeroM,oneM,),axis = 1)
a4=np.concatenate((oneM,oneM,oneM,zeroM,),axis = 1)

RDM_model_yindiao = np.concatenate((a1,a2,a3,a4),axis=0)

plot_rdm(RDM_model_yindiao, percentile=True)

0

In [33]:
# Calculate the representational similarity between 200ms and all the time points
corrs_EEGvsYindiao = rdms_corr(RDM_model_yindiao, rdms)

from neurora.rsa_plot import plot_tbytsim_withstats

#配合RDMS的时间窗是 50ms，步长是5ms = 0.005
ps_EEGvsYindiao = plot_tbytsim_withstats(corrs_EEGvsYindiao, start_time=-0.15, end_time=0.805, time_interval=0.005, p=0.05,clusterp=0.05,stats_time=[0,0.8], xlim=(-0.2,1), fontsize=9)



Computing similarities

Computing finished!

Significant time-windows:


In [14]:
np.save('./脑电结果/corrs_EEGvsYindiao_DD_颞叶.npy', corrs_EEGvsYindiao)
np.save('./脑电结果/ps_EEGvsYindiao_DD_颞叶.npy', ps_EEGvsYindiao)

## fo模型

In [ ]:
'''

更精细的构造方法 slope和height

'''

import math
import numpy as np

df = pd.read_excel('E:\实验\阅读困难\数据处理\pre2beishi_RSA_ASSR46\RSA\刺激space\\f0的slope和height.xlsx')

RDM_model_double = np.zeros([20,20], dtype=np.float32)
for i in range(len(df)):

    height1 = df['height'][i]
    slope1 = df['slope'][i]

    for j in range(len(df)):

        height2= df['height'][j]
        slope2 = df['slope'][j]

        '''
        计算公式
        '''
        distance = math.sqrt(math.pow((height2-height1),2)+math.pow((slope2-slope1),2))
        # print(distance)

        '''
        填写到合适的矩阵位置上。
        '''
        RDM_model_double[i][j] = distance
plot_rdm(RDM_model_double, percentile=True)

In [ ]:

corrs_f0 = rdms_corr(RDM_model_double, rdms)

#配合RDMS的时间窗是 50ms，步长是5ms = 0.005
plot_tbytsim_withstats(corrs_f0, start_time=-0.15, end_time=0.805, time_interval=0.005, p=0.1,clusterp=0.1,stats_time=[0,0.8])


## 行为模型

In [34]:
'''
基于行为数据的model RDM
'''

from scipy.io import loadmat

# 指定要读取的 .mat 文件路径
file_path_tone = "E:\实验\阅读困难\数据处理\pre4\RSA\models矩阵\\toneRDM.mat"  # 将 "path/to/your/file.mat" 替换为实际的文件路径
# 使用 loadmat 函数读取 .mat 文件
RDM_model_yindiao_xw = loadmat(file_path_tone)['final_rdm']

# 指定要读取的 .mat 文件路径
file_path_consonat = "E:\实验\阅读困难\数据处理\pre4\RSA\models矩阵\\consonantRDM.mat"  # 将 "path/to/your/file.mat" 替换为实际的文件路径
# 使用 loadmat 函数读取 .mat 文件
RDM_model_fuyin_xw = loadmat(file_path_consonat)['new_rdm']

# plot_rdm(RDM_model_yindiao_xw, percentile=True)
plot_rdm(RDM_model_fuyin_xw, percentile=True)

0

In [35]:
# Calculate the representational similarity between 200ms and all the time points
corrs_EEGvsFuyin_behavior = rdms_corr(RDM_model_fuyin_xw, rdms)
#配合RDMS的时间窗是 50ms，步长是5ms = 0.005
ps_corrs_EEGvsFuyin_behavior = plot_tbytsim_withstats(corrs_EEGvsFuyin_behavior, start_time=-0.15, end_time=0.805, time_interval=0.005, p=0.05,clusterp=0.05,stats_time=[0,0.8], xlim=[-0.2,1], fontsize=9, color='b')


Computing similarities

Computing finished!

Permutation test
Calculating: [====================================================================================================] 100.00%
Cluster-based permutation test finished!


Significant time-windows:
290ms to 420ms
515ms to 680ms
705ms to 800ms


In [36]:
from neurora.rsa_plot import plot_tbytsim_withstats
corrs_EEGvsYindiao_behavior = rdms_corr(RDM_model_yindiao_xw, rdms)
#配合RDMS的时间窗是 50ms，步长是5ms = 0.005
ps_corrs_EEGvsYindiao_behavior = plot_tbytsim_withstats(corrs_EEGvsYindiao_behavior, start_time=-0.15, end_time=0.805, time_interval=0.005, p=0.05,clusterp=0.05, color='r',stats_time=[0,0.8], xlim=[-0.2,1], fontsize=9)



Computing similarities

Computing finished!

Significant time-windows:


In [ ]:
np.save('./脑电结果/corrs_EEGvsFuyin_behavior_DD.npy', corrs_EEGvsFuyin_behavior)
np.save('./脑电结果/ps_corrs_EEGvsFuyin_behavior_DD.npy', ps_corrs_EEGvsFuyin_behavior)


# RSA和行为数据做相关


In [ ]:
import nibabel as nib
import numpy as np
import os
import math
from scipy.stats import spearmanr, pearsonr, kendalltau, ttest_1samp, ttest_rel
from skimage.measure import label
import sys
from neurora.stuff import get_cluster_index_1d_1sided
from neurora.stuff import show_progressbar
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
from scipy import signal
from scipy.stats import ttest_1samp
from nilearn import plotting, datasets, surface
import nibabel as nib
from neurora.stuff import get_affine, correct_by_threshold, get_bg_ch2, get_bg_ch2bet, \
    clusterbased_permutation_1d_1samp_1sided, clusterbased_permutation_2d_1samp_1sided
from decimal import Decimal

behavior_data = [1588.5,1605.46,1460.06,1844.59,543.99,53.11,2389.72,437.71, 1051.18, 2320.96,2132.45,2124.64,  652.25, 2491.82, 1663.75, 1663.75, 418.07]
def clusterbased_permutation_1d_1samp_1sided2(results, behavior_data=behavior_data, p_threshold=0.05, iter=5000, level=0):
    """
    1-sample & 1-sided cluster based permutation test for 2-D results

    Parameters
    ----------
    similarities : array
        A similarity matrix.
        The shape of similarities should be [n_subs, x, 0]. n_subs represents the number of subjects.
    behavior_data : array
        The behavior data for correlation calculation.
    p_threshold : float. Default is 0.05.
        The threshold of p-values.
    iter : int. Default is 5000.
        The times for iteration.

    Returns
    -------
    ps : float
        The permutation test results, p-values.
        The shape of ps is [x]. The values in ps should be 0 or 1, which represent not significant point or significant
        point after cluster-based permutation test, respectively.
    """
    nsubs, x, _ = np.shape(similarities)

    ps = np.zeros([x])
    ts = np.zeros([x])
    for t in range(x):
        time_point_data = similarities[:, t, 0]
        ts[t], p = pearsonr(behavior_data, time_point_data)
        if p < p_threshold and ts[t] > 0:
            ps[t] = 1
        else:
            ps[t] = 0

    cluster_index, cluster_n = get_cluster_index_1d_1sided(ps)
    print(cluster_n)

    if cluster_n != 0:
        cluster_ts = np.zeros([cluster_n])
        for i in range(cluster_n):
            for t in range(x):
                if cluster_index[t] == i + 1:
                    cluster_ts[i] = cluster_ts[i] + ts[t]

        permu_ts = np.zeros([iter])
        chance = np.full([nsubs], level)
        print("\nPermutation test")

        for i in range(iter):
            permu_cluster_ts = np.zeros([cluster_n])
            for j in range(cluster_n):
                for t in range(x):
                    if cluster_index[t] == j + 1:
                        v = np.hstack((results[:, t], chance))
                        vshuffle = np.random.permutation(v)
                        v1 = vshuffle[:nsubs]
                        v2 = vshuffle[nsubs:]
                        permu_cluster_ts[j] = permu_cluster_ts[j] + ttest_rel(v1, v2, alternative="greater")[0]
            permu_ts[i] = np.max(permu_cluster_ts)
            show_progressbar("Calculating", (i+1)*100/iter)
            if i == (iter - 1):
                print("\nCluster-based permutation test finished!\n")

        for i in range(cluster_n):
            index = 0
            for j in range(iter):
                if cluster_ts[i] > permu_ts[j]:
                    index = index + 1
            if index < iter * (1-p_threshold):
                for t in range(x):
                    if cluster_index[t] == i + 1:
                        ps[t] = 0

    return ps

def plot_tbytsim_withstats(similarities, start_time=0, end_time=1, smooth=True, p=0.05, cbpt=True, color='r',
                           lim=[-0.1, 0.8], figsize=[6.4, 3.6], x0=0, fontsize=16):

    """
    Plot the time-by-time Similarities with statistical results

    Parameters
    ----------
    similarities : array
        The Similarities.
        The size of similarities should be [n_subs, n_ts] or [n_subs, n_ts, 2]. n_subs, n_ts represent the number of
        subjects and number of time-points. 2 represents the similarity and a p-value.
    start_time : int or float. Default is 0.
        The start time.
    end_time : int or float. Default is 1.
        The end time.
    smooth : bool True or False. Default is True.
        Smooth the results or not.
    p : float. Default is 0.05.
        The threshold of p-values.
    cbpt : bool True or False. Default is True.
        Conduct cluster-based permutation test or not.
    color : matplotlib color or None. Default is 'r'.
        The color for the curve.
    lim : array or list [min, max]. Default is [-0.1, 0.8].
        The corrs view lims.
    figsize : array or list, [size_X, size_Y]. Default is [6.4, 3.6].
        The size of the figure.
    x0 : float. Default is 0.
        The Y-axis is at x=x0.
    fontsize : int or float. Default is 16.
        The fontsize of the labels.
    """

    if len(np.shape(similarities)) < 2 or len(np.shape(similarities)) > 3:

        return "Invalid input!"

    n = len(np.shape(similarities))

    minlim = lim[0]
    maxlim = lim[1]

    if n == 3:
        similarities = similarities[:, :, 0]

    nsubs = np.shape(similarities)[0]
    nts = np.shape(similarities)[1]

    tstep = float((end_time-start_time)/nts)

    if smooth is True:
        for sub in range(nsubs):
            for t in range(nts):

                if t<=1:
                    similarities[sub, t] = np.average(similarities[sub, :t+3])
                if t>1 and t<(nts-2):
                    similarities[sub, t] = np.average(similarities[sub, t-2:t+3])
                if t>=(nts-2):
                    similarities[sub, t] = np.average(similarities[sub, t-2:])

    avg = np.average(similarities, axis=0)
    err = np.zeros([nts], dtype=np.float)

    for t in range(nts):
        err[t] = np.std(similarities[:, t], ddof=1)/np.sqrt(nsubs)

    if cbpt == True:
        ps = clusterbased_permutation_1d_1samp_1sided2(similarities, level=0, p_threshold=p)
    else:
        ps = np.zeros([nts])
        for t in range(nts):
            ps[t] = ttest_1samp(similarities[:, t], 0, alternative="greater")[1]
            if ps[t] < p:
                ps[t] = 1
            else:
                ps[t] = 0

    for t in range(nts):
        if ps[t] == 1:
            plt.plot(t*tstep+start_time, (maxlim-minlim)*0.9+minlim, 's', color=color, alpha=1)
            xi = [t*tstep+start_time, t*tstep+tstep+start_time]
            ymin = [0]
            ymax = [avg[t]-err[t]]
            plt.fill_between(xi, ymax, ymin, facecolor=color, alpha=0.1)

    fig = plt.gcf()
    fig.set_size_inches(figsize[0], figsize[1])

    ax = plt.gca()
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)
    ax.spines["left"].set_linewidth(3)
    ax.spines["left"].set_position(("data", x0))
    ax.spines["bottom"].set_linewidth(3)
    ax.spines['bottom'].set_position(('data', 0))

    x = np.arange(start_time+0.5*tstep, end_time+0.5*tstep, tstep)
    plt.fill_between(x, avg + err, avg - err, facecolor=color, alpha=0.8)
    plt.ylim(minlim, maxlim)
    plt.xlim(start_time, end_time)
    plt.tick_params(labelsize=12)
    plt.xlabel("Time (s)", fontsize=fontsize)
    plt.ylabel("Representational Similarity", fontsize=fontsize)
    plt.show()

    return 0


ps_testBehave = clusterbased_permutation_1d_1samp_1sided2(results=corrs1[:, :, 0], behavior_data=behavior_data, p_threshold=0.05, iter=1000)

plot_tbytsim_withstats(corrs2)

In [ ]:
'''

这是想在RSA的基础上再封装一次.
用RSA每个时间点上的相关系数 和 行为数据做一次相关,
相当于用行为数据扫一遍

其实不应该扫,直接拿RSA显著的区域的值--峰值 来做一次相关就可以了.

'''

import numpy as np
from scipy.stats import pearsonr

# 行为数据（一维数组，17个值）
behavior_data = [1588.5,1605.46,1460.06,1844.59,543.99,53.11,2389.72,437.71,-2438.1, 1051.18, -973.79, -2204.05,-2295.48,2320.96,2132.45,2124.64, -2228.45, -694.64, 652.25, 2491.82, 1663.75, 1663.75, 418.07]
behavior_data = [1588.5,1605.46,1460.06,1844.59,543.99,53.11,2389.72,437.71, 1051.18, 2320.96,2132.45,2124.64,  652.25, 2491.82, 1663.75, 1663.75, 418.07]
# similarities数据（17个被试*191个时间点*2）
similarities = corrs_EEGvsFuyin_behavior

# 创建一个空的191x2矩阵来存储相关系数和p值
correlation_matrix = np.zeros((191, 2))

# 遍历每个时间点
for t in range(191):
    # 提取当前时间点上的17个数据
    time_point_data = similarities[:, t, 0]

    # 计算行为数据和当前时间点数据的相关系数和p值
    correlation_coefficient, p_value = pearsonr(behavior_data, time_point_data)

    # 将相关系数和p值填入矩阵
    correlation_matrix[t, 0] = correlation_coefficient
    correlation_matrix[t, 1] = p_value


In [37]:
import mne_rsa
